<a href="https://colab.research.google.com/github/samikshaa0604/FUTURE_ML_01/blob/main/samiksha1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FINAL CODE FOR ML_01 AI-Powered Sales Forecasting Dashboard



In [5]:
from prophet import Prophet


In [6]:
# Cell 1: Install Prophet (if not already installed)
!pip uninstall -y prophet
!pip uninstall -y cmdstanpy
!pip install prophet

Found existing installation: prophet 1.2.1
Uninstalling prophet-1.2.1:
  Successfully uninstalled prophet-1.2.1
Found existing installation: cmdstanpy 1.3.0
Uninstalling cmdstanpy-1.3.0:
  Successfully uninstalled cmdstanpy-1.3.0
  Using cached prophet-1.2.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.5 kB)
  Using cached cmdstanpy-1.3.0-py3-none-any.whl.metadata (4.2 kB)
Using cached prophet-1.2.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (12.1 MB)
Using cached cmdstanpy-1.3.0-py3-none-any.whl (99 kB)


In [7]:
# Cell 2: Import libraries
import pandas as pd
from prophet import Prophet
from google.colab import files

In [8]:
import pandas as pd

# Load dataset (make sure you uploaded the CSV first)
df = pd.read_csv('/content/Sample - Superstore (2).csv', encoding='latin1')

# Convert dates
df['Order Date'] = pd.to_datetime(df['Order Date'])

df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [9]:
# Cell 4: Create year and month features
df['year'] = df['Order Date'].dt.year
df['month'] = df['Order Date'].dt.month


In [10]:
# Cell 5: Aggregate data at required level
agg_df = df.groupby(
    ['year','month','Region','Category','Sub-Category','Product Name'],
    as_index=False
).agg({
    'Sales':'sum',
    'Profit':'sum',
    'Quantity':'sum'
})

agg_df.rename(columns={
    'Region':'region',
    'Category':'category',
    'Sub-Category':'sub_category',
    'Product Name':'product_name',
    'Sales':'total_sales',
    'Profit':'total_profit',
    'Quantity':'quantity'
}, inplace=True)


In [11]:
# Cell 6: Prepare data for forecasting (overall sales by month)
monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({'Sales':'sum'}).reset_index()
monthly_sales.rename(columns={'Order Date':'ds','Sales':'y'}, inplace=True)


/tmp/ipython-input-2369101067.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({'Sales':'sum'}).reset_index()


In [12]:
# Cell 7: Fit Prophet model
model = Prophet(interval_width=0.95)
model.fit(monthly_sales)

future = model.make_future_dataframe(periods=6, freq='M')  # 6 months forecast
forecast = model.predict(future)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [13]:
# Cell 8: Merge forecast with agg_df
# Extract year and month from forecast
forecast['year'] = forecast['ds'].dt.year
forecast['month'] = forecast['ds'].dt.month

forecast = forecast[['year','month','yhat','yhat_lower','yhat_upper']]
forecast.rename(columns={
    'yhat':'forecast_sales',
    'yhat_lower':'forecast_lower_ci',
    'yhat_upper':'forecast_upper_ci'
}, inplace=True)

# Add actual sales
actual = monthly_sales.copy()
actual['year'] = actual['ds'].dt.year
actual['month'] = actual['ds'].dt.month
actual = actual[['year','month','y']]
actual.rename(columns={'y':'actual_sales'}, inplace=True)

# Merge forecast + actual into agg_df
final_df = agg_df.merge(actual, on=['year','month'], how='left')
final_df = final_df.merge(forecast, on=['year','month'], how='left')


In [14]:
# Cell 9: Save and download final CSV
final_df.to_csv('final_output.csv', index=False)
files.download('final_output.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>